In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np

from tika import parser
from nltk.corpus import stopwords
import re   


In [5]:
tf.__version__

'2.15.0'

In [9]:
def review_to_words( raw_review ):
    review_text = parser.from_buffer("<html>" + raw_review + "</html>")["content"]                                                     #1
 
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)                #2
    words = letters_only.lower().split()                                #3
    
    stops = set(stopwords.words("english"))                             #4
    meaningful_words = [w for w in words if not w in stops]             #5
    return( " ".join( meaningful_words ))   

In [12]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
 
train = pd.read_csv("labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)                           #1
print(train.shape)                                                       #2
print(train.columns.values)                                              #2
 
num_reviews = train["review"].size                                       #3
clean_train_reviews = []                                                 #4
 
for i in tqdm(range( 0, num_reviews )):                                  #5
    clean_train_reviews.append( review_to_words( train["review"][i] ) )

(25000, 3)
['id' 'sentiment' 'review']


/tmp/ipykernel_4915/1637166378.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range( 0, num_reviews )):                                  #5


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer           #1
vectorizer = CountVectorizer(analyzer = "word",   \                   #1
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 
 
train_data_features = vectorizer.fit_transform(clean_train_reviews)   #2
train_data_features = train_data_features.toarray()                   #3
print(train_data_features.shape)              

In [ ]:
vocab = vectorizer.get_feature_names()         #1
print("size %d %s " % (len(vocab), vocab))     #1
 
dist = np.sum(train_data_features, axis=0)     #2
 
for tag, count in zip(vocab, dist):            #3
    print("%d, %s" % (count, tag))             #3
 
plt.scatter(vocab[0:99], dist[0:99])           #4
plt.xticks(vocab[0:99], rotation='vertical')   #4
plt.show()

In [ ]:
saver = tf.train.Saver() # A class that saves and restores  variables  from checkpoints
with tf.Session() as sess: # A session is a class that owns resources on computer, needs to run "with" so that session closes and resources are freed up
    sess.run(tf.global_variables_initializer()) # runs operations and evaluates tensors
    prev_err = 0.     #0. is a float                                                   
    for epoch in tqdm(range(training_epochs)): # creates the progress bar
        feed_dict = {} # initializes dictionary to store 
        for i in range(train_data_features.shape[1]): 
            feed_dict[Xs["X"+str(i)]] = train_data_features[:,i,None].reshape(len(train_data_features)) # puts stuff from train_data_features into new dictionary
        feed_dict[Y] = ys # set Y in feed dict to the train sentiment values                                                #3
        err, _ = sess.run([cost, train_op], feed_dict=feed_dict) # trains or something, outputs an error
        print(epoch, err)
        if abs(prev_err - err) < 0.0001:    # if difference between error and previous error is less than 0.0001, break                          #4
            break                                                         #4
        prev_err = err # otherwise, set prev_err to err
    
    w_val = sess.run(w, feed_dict)   # w_val inputting logistic regression weights
    save_path = saver.save(sess, "./en-netflix-binary-sentiment.ckpt")    # saves the model
 
print(w_val) # prints the weights
print(np.max(w_val)) # prints the max weight